In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.cluster import KMeans
from scipy.stats import norm
from sklearn.mixture import GaussianMixture



In [ ]:
# 1. Calculate average DNA methylation level
def average_methylation(methylation_data, start, end):
    region_data = methylation_data[(methylation_data['position'] >= start) & 
                                   (methylation_data['position'] <= end)]
    return region_data['methylation'].mean()



In [ ]:
# 2. Identify differentially methylated regions
def identify_dmrs(condition1, condition2, window_size=1000, step_size=100, p_threshold=0.05):
    dmrs = []
    for start in range(0, max(condition1['position'].max(), condition2['position'].max()), step_size):
        end = start + window_size
        region1 = condition1[(condition1['position'] >= start) & (condition1['position'] < end)]
        region2 = condition2[(condition2['position'] >= start) & (condition2['position'] < end)]
        if len(region1) > 0 and len(region2) > 0:
            t_stat, p_value = stats.ttest_ind(region1['methylation'], region2['methylation'])
            if p_value < p_threshold:
                dmrs.append((start, end, t_stat, p_value))
    return pd.DataFrame(dmrs, columns=['start', 'end', 't_statistic', 'p_value'])



In [ ]:
# 3. Visualize DNA methylation patterns
def plot_methylation_heatmap(methylation_data, genes, window=1000):
    matrix = []
    for gene in genes:
        start = gene['start'] - window
        end = gene['end'] + window
        region_data = methylation_data[(methylation_data['position'] >= start) & 
                                       (methylation_data['position'] <= end)]
        matrix.append(region_data['methylation'].values)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(matrix, cmap='coolwarm', center=0.5)
    plt.title('DNA Methylation Patterns')
    plt.xlabel('Position relative to gene')
    plt.ylabel('Genes')
    plt.show()



In [ ]:
# 4. Classify CpG islands
def classify_cpg_islands(cpg_data, threshold_low=0.3, threshold_high=0.7):
    classifications = []
    for _, cpg in cpg_data.iterrows():
        if cpg['mean_methylation'] < threshold_low:
            classifications.append('Hypomethylated')
        elif cpg['mean_methylation'] > threshold_high:
            classifications.append('Hypermethylated')
        else:
            classifications.append('Intermediate')
    return pd.Series(classifications, index=cpg_data.index)




In [ ]:
# 5. Integrate methylation and gene expression data
def integrate_methylation_expression(methylation_data, expression_data, window=1000):
    integrated_data = []
    for gene, expr in expression_data.items():
        gene_methylation = methylation_data[(methylation_data['position'] >= gene['start'] - window) & 
                                            (methylation_data['position'] <= gene['end'] + window)]
        mean_methylation = gene_methylation['methylation'].mean()
        integrated_data.append((gene, mean_methylation, expr))
    return pd.DataFrame(integrated_data, columns=['gene', 'mean_methylation', 'expression'])



In [ ]:
# 6. Peak calling for ChIP-seq data
def call_peaks(chip_data, control_data, window_size=1000, fold_enrichment=4, p_threshold=1e-5):
    peaks = []
    for start in range(0, chip_data['position'].max(), window_size):
        end = start + window_size
        chip_counts = chip_data[(chip_data['position'] >= start) & (chip_data['position'] < end)]['counts'].sum()
        control_counts = control_data[(control_data['position'] >= start) & (control_data['position'] < end)]['counts'].sum()
        
        if chip_counts > 0 and control_counts > 0:
            fold_change = chip_counts / control_counts
            p_value = stats.poisson.sf(chip_counts, control_counts)
            
            if fold_change >= fold_enrichment and p_value <= p_threshold:
                peaks.append((start, end, fold_change, p_value))
    
    return pd.DataFrame(peaks, columns=['start', 'end', 'fold_change', 'p_value'])



In [ ]:
# 7. Calculate histone modification enrichment around TSS
def histone_enrichment_tss(histone_data, tss_positions, window=5000):
    enrichment = np.zeros(2 * window + 1)
    for tss in tss_positions:
        region_data = histone_data[(histone_data['position'] >= tss - window) & 
                                   (histone_data['position'] <= tss + window)]
        region_data['relative_position'] = region_data['position'] - tss + window
        enrichment += np.bincount(region_data['relative_position'], 
                                  weights=region_data['signal'], 
                                  minlength=2*window+1)
    return enrichment / len(tss_positions)



In [ ]:
# 8. Identify bivalent chromatin domains
def identify_bivalent_domains(activating_marks, repressing_marks, window_size=5000):
    bivalent_domains = []
    for start in range(0, max(activating_marks['position'].max(), repressing_marks['position'].max()), window_size):
        end = start + window_size
        activating = activating_marks[(activating_marks['position'] >= start) & (activating_marks['position'] < end)]
        repressing = repressing_marks[(repressing_marks['position'] >= start) & (repressing_marks['position'] < end)]
        
        if activating['signal'].sum() > 0 and repressing['signal'].sum() > 0:
            bivalent_domains.append((start, end))
    
    return pd.DataFrame(bivalent_domains, columns=['start', 'end'])



In [ ]:
# 9. Predict enhancer regions
def predict_enhancers(h3k4me1_data, h3k27ac_data, window_size=1000, threshold=0.8):
    enhancers = []
    for start in range(0, max(h3k4me1_data['position'].max(), h3k27ac_data['position'].max()), window_size):
        end = start + window_size
        h3k4me1 = h3k4me1_data[(h3k4me1_data['position'] >= start) & (h3k4me1_data['position'] < end)]
        h3k27ac = h3k27ac_data[(h3k27ac_data['position'] >= start) & (h3k27ac_data['position'] < end)]
        
        if h3k4me1['signal'].mean() > threshold and h3k27ac['signal'].mean() > threshold:
            enhancers.append((start, end))
    
    return pd.DataFrame(enhancers, columns=['start', 'end'])



In [ ]:
# 10. Simulate DNA methylation data
def simulate_methylation_data(n_regions=1000, n_cpgs_per_region=20):
    methylation_data = []
    for i in range(n_regions):
        start = i * 1000
        cpg_positions = np.sort(np.random.choice(range(start, start+1000), n_cpgs_per_region, replace=False))
        methylation_levels = np.random.beta(1, 1, n_cpgs_per_region)
        for pos, meth in zip(cpg_positions, methylation_levels):
            methylation_data.append((pos, meth))
    
    return pd.DataFrame(methylation_data, columns=['position', 'methylation'])



In [ ]:
# 11. Implement a function to perform peak calling on ATAC-seq data and visualize the distribution of peaks relative to transcription start sites.
import pybedtools
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

def call_atac_peaks(bam_file, genome, output_file):
    """
    Perform peak calling on ATAC-seq data using MACS2.
    
    Parameters:
    bam_file (str): Path to the BAM file containing aligned ATAC-seq reads
    genome (str): Genome size (e.g., 'hs' for human, 'mm' for mouse)
    output_file (str): Path to save the called peaks
    
    Returns:
    pybedtools.BedTool: Called peaks
    """
    peaks = pybedtools.BedTool(bam_file).sort().bam_to_bed().slop(g=genome, b=100).merge()
    peaks = peaks.filter(lambda x: int(x.end) - int(x.start) > 50)
    peaks.saveas(output_file)
    return peaks

def visualize_peak_distribution(peaks, tss_file, output_file):
    """
    Visualize the distribution of ATAC-seq peaks relative to transcription start sites.
    
    Parameters:
    peaks (pybedtools.BedTool): Called ATAC-seq peaks
    tss_file (str): Path to the BED file containing transcription start sites
    output_file (str): Path to save the output plot
    """
    tss = pybedtools.BedTool(tss_file)
    distances = []
    
    for peak in peaks:
        nearest_tss = tss.closest(peak, d=True)[0]
        distance = int(nearest_tss[-1])
        if nearest_tss.strand == '-':
            distance = -distance
        distances.append(distance)
    
    distances = np.array(distances)
    distances = distances[np.abs(distances) <= 5000]  # Limit to +/- 5kb
    
    plt.figure(figsize=(10, 6))
    density = gaussian_kde(distances)
    xs = np.linspace(-5000, 5000, 200)
    plt.plot(xs, density(xs))
    plt.title('Distribution of ATAC-seq Peaks Relative to TSS')
    plt.xlabel('Distance to TSS (bp)')
    plt.ylabel('Density')
    plt.axvline(x=0, color='r', linestyle='--', label='TSS')
    plt.legend()
    plt.savefig(output_file)
    plt.close()

# Example usage
bam_file = "atac_seq_aligned.bam"
genome = "hg38"
peak_file = "atac_peaks.bed"
tss_file = "transcription_start_sites.bed"

peaks = call_atac_peaks(bam_file, genome, peak_file)
visualize_peak_distribution(peaks, tss_file, "atac_peak_distribution.png")

In [ ]:
# 12. Create a script to analyze Hi-C data and identify topologically associating domains (TADs) using the insulation score method.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cooler

def calculate_insulation_score(matrix, window_size=5):
    """
    Calculate the insulation score for a Hi-C contact matrix.
    
    Parameters:
    matrix (np.array): Hi-C contact matrix
    window_size (int): Size of the sliding window
    
    Returns:
    np.array: Insulation scores
    """
    n = matrix.shape[0]
    scores = np.zeros(n)
    
    for i in range(window_size, n - window_size):
        window = matrix[i-window_size:i+window_size, i-window_size:i+window_size]
        scores[i] = np.mean(window)
    
    return scores

def call_tads(insulation_scores, min_size=5):
    """
    Call TADs based on insulation scores.
    
    Parameters:
    insulation_scores (np.array): Insulation scores
    min_size (int): Minimum size of a TAD in bins
    
    Returns:
    list: List of TAD boundaries
    """
    local_minima = (insulation_scores[1:-1] < insulation_scores[:-2]) & \
                   (insulation_scores[1:-1] < insulation_scores[2:])
    boundaries = np.where(local_minima)[0] + 1
    
    tads = []
    for i in range(len(boundaries) - 1):
        if boundaries[i+1] - boundaries[i] >= min_size:
            tads.append((boundaries[i], boundaries[i+1]))
    
    return tads

def analyze_hic_data(cool_file, chromosome, resolution, output_file):
    """
    Analyze Hi-C data to identify TADs and visualize the results.
    
    Parameters:
    cool_file (str): Path to the .cool file containing Hi-C data
    chromosome (str): Chromosome to analyze
    resolution (int): Resolution of the Hi-C data in base pairs
    output_file (str): Path to save the output plot
    """
    # Load Hi-C data
    c = cooler.Cooler(cool_file)
    matrix = c.matrix(balance=True).fetch(chromosome)
    
    # Calculate insulation scores
    insulation_scores = calculate_insulation_score(matrix)
    
    # Call TADs
    tads = call_tads(insulation_scores)
    
    # Visualize results
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    
    # Plot Hi-C matrix
    im = ax1.imshow(np.log1p(matrix), cmap='YlOrRd', aspect='auto')
    ax1.set_title(f'Hi-C Contact Matrix - {chromosome}')
    plt.colorbar(im, ax=ax1, label='Log(Contact Frequency + 1)')
    
    # Plot insulation score
    ax2.plot(insulation_scores)
    ax2.set_title('Insulation Score')
    ax2.set_xlabel('Genomic Position (bins)')
    ax2.set_ylabel('Insulation Score')
    
    # Highlight TADs
    for start, end in tads:
        ax1.axvline(x=start, color='blue', linestyle='--', alpha=0.5)
        ax1.axvline(x=end, color='blue', linestyle='--', alpha=0.5)
        ax2.axvline(x=start, color='blue', linestyle='--', alpha=0.5)
        ax2.axvline(x=end, color='blue', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close()
    
    return tads

# Example usage
cool_file = "hic_data.cool"
chromosome = "chr1"
resolution = 25000  # 25 kb resolution
tads = analyze_hic_data(cool_file, chromosome, resolution, "hic_tad_analysis.png")

print(f"Number of TADs identified: {len(tads)}")

In [ ]:
# 13. Develop a program to calculate and visualize A/B compartments from Hi-C data using principal component analysis.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cooler
from sklearn.decomposition import PCA

def calculate_ab_compartments(cool_file, chromosome, resolution, output_file):
    """
    Calculate and visualize A/B compartments from Hi-C data using PCA.
    
    Parameters:
    cool_file (str): Path to the .cool file containing Hi-C data
    chromosome (str): Chromosome to analyze
    resolution (int): Resolution of the Hi-C data in base pairs
    output_file (str): Path to save the output plot
    
    Returns:
    np.array: First principal component (PC1) values
    """
    # Load Hi-C data
    c = cooler.Cooler(cool_file)
    matrix = c.matrix(balance=True).fetch(chromosome)
    
    # Calculate correlation matrix
    corr_matrix = np.corrcoef(matrix)
    corr_matrix = np.nan_to_num(corr_matrix)
    
    # Perform PCA
    pca = PCA(n_components=1)
    pc1 = pca.fit_transform(corr_matrix).flatten()
    
    # Ensure positive values correspond to A compartments
    if np.sum(pc1[pc1 > 0]) < np.sum(pc1[pc1 < 0]):
        pc1 = -pc1
    
    # Visualize results
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    
    # Plot Hi-C matrix
    im = ax1.imshow(np.log1p(matrix), cmap='YlOrRd', aspect='auto')
    ax1.set_title(f'Hi-C Contact Matrix - {chromosome}')
    plt.colorbar(im, ax=ax1, label='Log(Contact Frequency + 1)')
    
    # Plot PC1 values
    ax2.plot(pc1)
    ax2.set_title('A/B Compartments (PC1)')
    ax2.set_xlabel('Genomic Position (bins)')
    ax2.set_ylabel('PC1 Value')
    ax2.axhline(y=0, color='r', linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close()
    
    return pc1

# Example usage
cool_file = "hic_data.cool"
chromosome = "chr1"
resolution = 100000  # 100 kb resolution
pc1 = calculate_ab_compartments(cool_file, chromosome, resolution, "ab_compartments.png")

# Classify regions into A and B compartments
a_compartments = np.where(pc1 > 0)[0]
b_compartments = np.where(pc1 < 0)[0]

print(f"Number of A compartment regions: {len(a_compartments)}")
print(f"Number of B compartment regions: {len(b_compartments)}")

In [ ]:
# 14. Write a function to implement a basic epigenetic clock model using DNA methylation data and chronological age information.
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

def train_epigenetic_clock(methylation_data, ages, n_cpgs=100, test_size=0.2):
    """
    Train an epigenetic clock model using DNA methylation data.
    
    Parameters:
    methylation_data (pd.DataFrame): DNA methylation data (CpG sites x samples)
    ages (pd.Series): Chronological ages of the samples
    n_cpgs (int): Number of CpG sites to use in the model
    test_size (float): Proportion of data to use for testing
    
    Returns:
    tuple: Trained model, selected CpG sites, and evaluation metrics
    """
    # Select most variable CpG sites
    var_cpgs = methylation_data.var().nlargest(n_cpgs).index
    X = methylation_data[var_cpgs]
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, ages, test_size=test_size, random_state=42)
    
    # Train Random Forest model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate model
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # Visualize results
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    plt.xlabel('Chronological Age')
    plt.ylabel('Predicted Age')
    plt.title('Epigenetic Clock: Predicted vs Chronological Age')
    plt.text(0.05, 0.95, f'MAE: {mae:.2f}\nRMSE: {rmse:.2f}', transform=plt.gca().transAxes)
    plt.tight_layout()
    plt.savefig('epigenetic_clock.png')
    plt.close()
    
    return model, var_cpgs, {'MAE': mae, 'RMSE': rmse}

# Example usage
# Generate synthetic methylation data and ages
np.random.seed(42)
n_samples = 1000
n_cpgs = 10000
methylation_data = pd.DataFrame(np.random.beta(1, 1, size=(n_cpgs, n_samples)))
ages = pd.Series(np.random.uniform(20, 80, n_samples))

model, selected_cpgs, metrics = train_epigenetic_clock(methylation_data, ages)

print("Selected CpG sites:", selected_cpgs)
print("Model performance:")
print(f"MAE: {metrics['MAE']:.2f}")
print(f"RMSE: {metrics['RMSE']:.2f}")

# Predict age for new samples
new_samples = pd.DataFrame(np.random.beta(1, 1, size=(n_cpgs, 5)))
predicted_ages = model.predict(new_samples.loc[selected_cpgs])
print("Predicted ages for new samples:", predicted_ages)

In [ ]:
# 15. Implement a script to integrate chromatin accessibility data with long-range interaction data to identify potential enhancer-promoter interactions.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

def integrate_accessibility_interactions(accessibility_data, interaction_data, gene_annotations, output_file):
    """
    Integrate chromatin accessibility data with long-range interaction data to identify
    potential enhancer-promoter interactions.
    
    Parameters:
    accessibility_data (pd.DataFrame): ATAC-seq peak data (columns: chrom, start, end, score)
    interaction_data (pd.DataFrame): Hi-C interaction data (columns: chrom1, start1, end1, chrom2, start2, end2, score)
    gene_annotations (pd.DataFrame): Gene annotations (columns: chrom, start, end, gene_name, strand)
    output_file (str): Path to save the output plot
    
    Returns:
    pd.DataFrame: Potential enhancer-promoter interactions
    """
    # Function to check if two regions overlap
    def regions_overlap(start1, end1, start2, end2):
        return start1 <= end2 and start2 <= end1
    
    # Identify potential enhancers (ATAC-seq peaks not overlapping promoters)
    promoter_region = 2000  # Define promoter as 2kb upstream of TSS
    enhancers = []
    for _, peak in accessibility_data.iterrows():
        is_enhancer = True
        for _, gene in gene_annotations.iterrows():
            if gene['strand'] == '+':
                promoter_start, promoter_end = gene['start'] - promoter_region, gene['start']
            else:
                promoter_start, promoter_end = gene['end'], gene['end'] + promoter_region
            
            if regions_overlap(peak['start'], peak['end'], promoter_start, promoter_end):
                is_enhancer = False
                break
        
        if is_enhancer:
            enhancers.append(peak)
    
    enhancers = pd.DataFrame(enhancers)
    
    # Identify potential enhancer-promoter interactions
    potential_interactions = []
    for _, enhancer in enhancers.iterrows():
        for _, interaction in interaction_data.iterrows():
            if regions_overlap(enhancer['start'], enhancer['end'], interaction['start1'], interaction['end1']):
                for _, gene in gene_annotations.iterrows():
                    if regions_overlap(gene['start'], gene['end'], interaction['start2'], interaction['end2']):
                        potential_interactions.append({
                            'enhancer_chrom': enhancer['chrom'],
                            'enhancer_start': enhancer['start'],
                            'enhancer_end': enhancer['end'],
                            'enhancer_score': enhancer['score'],
                            'gene_name': gene['gene_name'],
                            'gene_start': gene['start'],
                            'gene_end': gene['end'],
                            'interaction_score': interaction['score']
                        })
    
    potential_interactions = pd.DataFrame(potential_interactions)
    
    # Visualize potential enhancer-promoter interactions
    plt.figure(figsize=(12, 8))
    plt.scatter(potential_interactions['enhancer_score'], potential_interactions['interaction_score'], alpha=0.5)
    plt.xlabel('Enhancer Accessibility Score')
    plt.ylabel('Interaction Score')
    plt.title('Potential Enhancer-Promoter Interactions')
    
    for _, interaction in potential_interactions.iterrows():
        plt.annotate(interaction['gene_name'], 
                     (interaction['enhancer_score'], interaction['interaction_score']),
                     xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close()
    
    return potential_interactions

# Example usage
accessibility_data = pd.DataFrame({
    'chrom': ['chr1', 'chr1', 'chr2', 'chr2'],
    'start': [1000, 5000, 2000, 8000],
    'end': [2000, 6000, 3000, 9000],
    'score': [10, 15, 8, 12]
})

interaction_data = pd.DataFrame({
    'chrom1': ['chr1', 'chr1', 'chr2'],
    'start1': [1500, 5500, 2500],
    'end1': [2500, 6500, 3500],
    'chrom2': ['chr1', 'chr1', 'chr2'],
    'start2': [10000, 15000, 12000],
    'end2': [11000, 16000, 13000],
    'score': [0.8, 0.6, 0.7]
})

gene_annotations = pd.DataFrame({
    'chrom': ['chr1', 'chr1', 'chr2'],
    'start': [9500, 14500, 11500],
    'end': [11500, 16500, 13500],
    'gene_name': ['Gene1', 'Gene2', 'Gene3'],
    'strand': ['+', '-', '+']
})

output_file = 'enhancer_promoter_interactions.png'

potential_interactions = integrate_accessibility_interactions(accessibility_data, interaction_data, gene_annotations, output_file)
print(potential_interactions)

In [ ]:
# Example usage:
# methylation_data = simulate_methylation_data()
# avg_methylation = average_methylation(methylation_data, 1000, 2000)

# condition1 = simulate_methylation_data()
# condition2 = simulate_methylation_data()
# dmrs = identify_dmrs(condition1, condition2)

# genes = [{'start': 1000, 'end': 2000}, {'start': 3000, 'end': 4000}]
# plot_methylation_heatmap(methylation_data, genes)

# cpg_islands = pd.DataFrame({'mean_methylation': np.random.uniform(0, 1, 100)})
# cpg_classifications = classify_cpg_islands(cpg_islands)

# expression_data = {'gene1': 10, 'gene2': 20}
# integrated_data = integrate_methylation_expression(methylation_data, expression_data)

# chip_data = pd.DataFrame({'position': range(10000), 'counts': np.random.poisson(10, 10000)})
# control_data = pd.DataFrame({'position': range(10000), 'counts': np.random.poisson(5, 10000)})
# peaks = call_peaks(chip_data, control_data)

# tss_positions = [1000, 3000, 5000]
# histone_data = pd.DataFrame({'position': range(10000), 'signal': np.random.poisson(5, 10000)})
# enrichment = histone_enrichment_tss(histone_data, tss_positions)

# activating_marks = pd.DataFrame({'position': range(10000), 'signal': np.random.poisson(5, 10000)})
# repressing_marks = pd.DataFrame({'position': range(10000), 'signal': np.random.poisson(5, 10000)})
# bivalent_domains = identify_bivalent_domains(activating_marks, repressing_marks)

# h3k4me1_data = pd.DataFrame({'position': range(10000), 'signal': np.random.uniform(0, 1, 10000)})
# h3k27ac_data = pd.DataFrame({'position': range(10000), 'signal': np.random.uniform(0, 1, 10000)})
# enhancers = predict_enhancers(h3k4me1_data, h3k27ac_data)

# simulated_methylation = simulate_methylation_data()